In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram
import meshcat

# Build plant

In [4]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()
# visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
#                                       delete_prefix_on_load=False)

# diagram = builder.Build()
# visualizer.load()


# Setup meshcat visualization

In [5]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


# Build Certified Iris Region Object

In [6]:
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_faces': -1
}

In [7]:
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)

In [8]:
seed_points = np.tan(np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ])    
                        /2)

regions, ellipses = iris_generator.iris_in_rational_space(seed_points)


0
snopt_example=[ 0.35529068 -0.22984731  0.02559963], growth = 0.05990553085918943
snopt_example=[ 0.28673366 -0.30128467  0.33402563], growth = 0.09485392272232375
snopt_example=[ 0.28776043 -0.30045108  0.32725976], growth = 0.09339195802962706
terminating because a required containment point would have not been contained
Time:   1.67 	Volume:  21.77 	Center: [1.44312073e-09 1.44280166e-09 1.44440504e-09]
0
snopt_example=[ 0.35529266 -0.22984424  0.02559962], growth = 0.05990553077553568
snopt_example=[ 0.30804713 -0.27858144  0.24207296], growth = 0.07703332293708175
terminating because a required containment point would have not been contained
Time:   1.01 	Volume:  21.77 	Center: [1.44312073e-09 1.44280166e-09 1.44440504e-09]
snopt_example=[ 0.30547805 -0.52068241  0.30020753], growth = 354169277.01944494
snopt_example=[ 0.19754042 -0.45436384  0.26846797], growth = 33224401.49490118
snopt_example=[ 0.19598983 -0.45413506  0.27106354], growth = 32414169.41665411
0
snopt_example=[

In [9]:
trajectory_start = np.tan(np.array([-1, -2, 0.5])/2) # seed_points[1,:]
trajectory_end = np.tan(np.array([0.25, -1.6, -0.25])/2)  #seed_points[3,:]

In [10]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions, ellipses)
    visualizer.plot_seedpoints(seed_points)
    visualizer.visualize_collision_constraint(N = 50)
    

## Certify the regions

In [11]:
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

Time to initialize region program: 1.0142
time to create Region Certification Problem: : 0.5986
Time to initialize region program: 1.0348
time to create Region Certification Problem: : 0.6304
Time to initialize region program: 1.2945
time to create Region Certification Problem: : 1.4097
Time to initialize region program: 1.4544
time to create Region Certification Problem: : 2.2925
Time to initialize region program: 1.1328
time to create Region Certification Problem: : 1.0335
Time to initialize region program: 1.0962
time to create Region Certification Problem: : 0.8584


{<pydrake.geometry.optimization.HPolyhedron at 0x7f5cc3ef52f0>: <certified_iris_generator.RegionCertificationProblem at 0x7f5d04601790>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f5cc3ef5230>: <certified_iris_generator.RegionCertificationProblem at 0x7f5d04601df0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f5cc3f0adf0>: <certified_iris_generator.RegionCertificationProblem at 0x7f5cc3e91850>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f5cc3f0adb0>: <certified_iris_generator.RegionCertificationProblem at 0x7f5cc3e91160>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f5cc3ef04f0>: <certified_iris_generator.RegionCertificationProblem at 0x7f5cc3e91220>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f5cc3f0ac70>: <certified_iris_generator.RegionCertificationProblem at 0x7f5cc3e91190>}

In [ ]:
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

Starting Region 1/6
min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.8660254  1.65860451 0.10832327 0.8660254  0.07344629 1.62372753]
max_eps = [1.73205081 3.31720903 0.21664655 1.73205081 0.14689259 3.24745507]
Biggest difference: 3.317209025457233
Smallest difference: 0.14689258966773977
Fixed eps is not success

min_eps = [0.8660254  1.65860451 0.10832327 0.8660254  0.07344629 1.62372753]
cur_eps = [1.29903811 2.48790677 0.16248491 1.29903811 0.11016944 2.4355913 ]
max_eps = [1.73205081 3.31720903 0.21664655 1.73205081 0.14689259 3.24745507]
Biggest difference: 1.6586045127286164
Smallest difference: 0.07344629483386989
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.62634388 1.24395338 0.08124245 0.46510824 0.05508472 1.21779565]
max_eps = [1.25268776 2.48790677 0.1624849  0.93021648 0.11016943 2.4355913 ]
Biggest difference: 2.487906768142779
Smallest difference: 0.11016943388782734
Fixed eps is not success

min_eps = [0.62634388 1.24395338 0.0812

Fixed eps is not success

min_eps = [0.99074381 2.4347021  0.15900987 0.25879608 0.10781327 2.38350541]
cur_eps = [0.99860686 2.45402513 0.16027185 0.26085001 0.10866893 2.40242212]
max_eps = [1.0064699  2.47334816 0.16153383 0.26290395 0.10952459 2.42133883]
Biggest difference: 0.0386460650410374
Smallest difference: 0.001711321708748284
Fixed eps is not success

min_eps = [0.99860686 2.45402513 0.16027185 0.26085001 0.10866893 2.40242212]
cur_eps = [1.00253838 2.46368665 0.16090284 0.26187698 0.10909676 2.41188047]
max_eps = [1.0064699  2.47334816 0.16153383 0.26290395 0.10952459 2.42133883]
Biggest difference: 0.0193230325205187
Smallest difference: 0.000855660854374149
Fixed eps is not success

min_eps = [1.00253838 2.46368665 0.16090284 0.26187698 0.10909676 2.41188047]
cur_eps = [1.00450414 2.4685174  0.16121834 0.26239047 0.10931067 2.41660965]
max_eps = [1.0064699  2.47334816 0.16153383 0.26290395 0.10952459 2.42133883]
Biggest difference: 0.00966151626025935
Smallest differenc

Fixed eps is not success

min_eps = [1.74436424 2.46914418 1.13060524 0.73288949 0.13371111 1.35528674]
cur_eps = [1.86896169 2.64551162 1.21136275 0.78523874 0.1432619  1.45209294]
max_eps = [1.99355913 2.82187906 1.29212027 0.83758799 0.1528127  1.54889913]
Biggest difference: 0.35273488293695543
Smallest difference: 0.019101587317838287
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.93448084 1.28740559 0.60568138 0.31621961 0.07163095 0.59687776]
max_eps = [1.86896169 2.57481119 1.21136275 0.63243922 0.1432619  1.19375551]
Biggest difference: 2.57481118686663
Smallest difference: 0.14326190491649007
Fixed eps is not success

min_eps = [0.93448084 1.28740559 0.60568138 0.31621961 0.07163095 0.59687776]
cur_eps = [1.40172126 1.93110839 0.90852207 0.47432941 0.10744643 0.89531664]
max_eps = [1.86896169 2.57481119 1.21136275 0.63243922 0.1432619  1.19375551]
Biggest difference: 1.287405593433315
Smallest difference: 0.07163095245824504
Fixed eps 

In [ ]:
# uncomment to visualize planes (TODO)
# visualizer.region_to_collision_pair_to_plane_dictionary = iris_generator.linesearch_regions_to_certificates_by_collision_pair_map

In [ ]:
do_alt_cert = False
if do_alt_cert:
    iris_generator.certify_and_adjust_regions_by_bilinear_alternation(max_iter = 1000, 
                                                                 stopped_improving_count_threshold = 5,
                                                                 not_improving_threshold = 1e-4)

In [ ]:
# plot new regions
if do_viz:
    if do_alt_cert:
        visualizer.plot_regions(iris_generator.alternation_search_regions, region_suffix = '_new_by_alternation')
    if do_linesearch_cert:
        visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')
    

In [ ]:
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra
# Solve path planning
start_time = time.time()
# trajectory through linesearch_regions
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(trajectory_start.squeeze(), trajectory_end.squeeze(), 
                                                iris_generator.linesearch_regions)
spp.set_max_velocity(0.3*np.ones(plant.num_positions()))
spp.set_extra_control_points_per_region(3)
traj_iris = spp.SolveVerbose()
visualizer.draw_traj_tspace(traj_iris, 100, 'iris')

In [ ]:
print(time.time() - start_time)
print(traj_iris.start_time())
print(traj_iris.end_time())
# draw_traj_tspace(traj_iris, 100, 'iris')
#animate Iris + spp sol
substeps = 1000
its = 10
visualizer.animate_t(traj_iris, substeps, its*substeps)